In [1]:
from selenium import webdriver
from PIL import Image
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains

In [14]:
options = webdriver.ChromeOptions() 
options.add_argument("download.default_directory=/Users/joel/library_")

driver = webdriver.Chrome('/Users/joel/selenium_drivers/chromedriver 3',
                          chrome_options=options)

/Users/joel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [20]:
def location(tweet, path, count = 0):
    location = tweet.location
    size = tweet.size
    driver.save_screenshot('{}-{}.png'.format(path, count))
    x = location['x']
    y = location['y']
    width = location['x']+size['width']
    height = location['y']+size['height']

    return x, y, width, height

def crop_screenshot(count, x, y, width, height, path):    
    im = Image.open('{}-{}.png'.format(path, count))

    im = im.crop((int(x), int(y), int(width), int(height)))
    im.save('{}-{}.png'.format(path, count))

def find_tweet(driver, url, path, count=0):
    driver.get(url)
    sleep(2)
    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div/div/section/div/div/div/div[2]/div/article/div/div/div[2]/div[2]/div[2]/div[1]').click()
    tweets = driver.find_elements_by_tag_name('article')
    username = tweets[0].text.split('\n')[0]
    x, y, width, height = location(tweets[0], path, count)
    
    return tweets, username, x, y, width, height

def thread_screenshot(tweet, count, x, y, width, height, username, path):
    try:
        tweet.find_element_by_xpath('//*[@id="tweet-text"]/span')
    except:
        driver.execute_script("scrollBy(0,250);")
        sleep(1)
        driver.execute_script("scrollBy(0,-250);")
        sleep(1)
        driver.execute_script("scrollBy(0,250);")
        tweet.click()
    sleep(2)
    driver.save_screenshot('{}-{}.png'.format(path, count))
    crop_screenshot(count, x, y, width, height, path)

    try:
        found = driver.find_elements_by_tag_name('article')
        tweet_list = [element for element in found if element.text.split('\n')[0] == username]
    except:
        sleep(1)
        found = driver.find_elements_by_tag_name('article')
        tweet_list = [element for element in found if element.text.split('\n')[0] == username]
    
    for tweet_ in tweet_list:
        split = tweet_.text.split('\n')
        if 'Replying to ' in split:
            thread_screenshot(tweet_, count+1, x, y, width, height, username, path)
            break

def twitter_thread(driver, url, path):
    
    tweets, username, x, y, width, height = find_tweet(driver, url, path)
    
    thread_screenshot(tweets[0], 0, x, y, width, height, username, path)

In [21]:
url = 'https://twitter.com/choo_ek/status/1242323621705478144?s=20'
path = '/Users/joel/library_/triage'

In [22]:
twitter_thread(driver, url, path)

In [15]:
driver.get(url)

In [17]:
article = driver.find_element_by_tag_name('article')
driver.find_element_by_class_name('css-1dbjc4n').click()